# Spectral-fPINN for the Fractional Bratu Equation

This notebook implements the **Physics-Informed Neural Network (PINN)** framework from the paper:

*"Physics-Informed Neural Networks for Solving Fractional Bratu Ordinary Differential Equations with Hermite and Gegenbauer Basis Functions"*

## Problem

We solve the **fractional Bratu equation**:
$$D^\alpha u(x) + \lambda e^{u(x)} = 0, \quad x \in [0, 1], \quad 0 < \alpha \leq 1$$
with **boundary conditions** $u(0) = 0$ and $u(1) = 0$, where $D^\alpha$ is the **Caputo fractional derivative**.

## Method

- Approximate $u(x)$ with a neural network $u_\theta(x)$.
- Project the network output onto a **spectral basis** (Hermite or Gegenbauer) to compute $D^\alpha u$ via an **operational matrix** (no automatic differentiation for fractional derivatives).
- Train by minimizing the physics residual and boundary loss.

## 1. Imports and device

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from scipy.special import gamma, eval_hermite, eval_gegenbauer
from numpy.polynomial import hermite as np_hermite
import matplotlib.pyplot as plt
from time import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

: 

## 2. Caputo fractional derivative of monomials

For polynomial basis functions we use:
$$D^\alpha x^k = \frac{\Gamma(k+1)}{\Gamma(k+1-\alpha)} x^{k-\alpha}, \quad k \geq \lceil \alpha \rceil$$

In [ ]:
def caputo_monomial(x, k, alpha):
    """Caputo fractional derivative of x^k at point(s) x. x can be array."""
    x = np.atleast_1d(np.asarray(x, dtype=float))
    k_ceil = int(np.ceil(alpha))
    if k < k_ceil:
        return np.zeros_like(x)
    coef = gamma(k + 1) / gamma(k + 1 - alpha)
    return coef * (x ** (k - alpha))

## 3. Basis functions: Hermite and Gegenbauer

- **Hermite** (physicist): $H_{n+1}(x) = 2x H_n(x) - 2n H_{n-1}(x)$, evaluated on $[0,1]$.
- **Gegenbauer** $C_n^{(\lambda)}(t)$ on $[-1,1]$; we map $x \in [0,1]$ to $t = 2x - 1$.

In [ ]:
def hermite_poly_coeffs(n):
    """Monomial coefficients (ascending power) of physicist Hermite polynomial H_n."""
    H = np_hermite.Hermite.basis(n)
    # Hermite in Hermite basis -> convert to standard polynomial (ascending)
    c_herm = H.coef
    # numpy polynomial Hermite: H_n(x) with recurrence. Get coeffs in monomial form.
    p = np_hermite.herm2poly(c_herm)
    return np.asarray(p, dtype=float)

def eval_phi_hermite(x, n):
    """Evaluate Hermite polynomial H_n at x (can be array)."""
    return eval_hermite(n, np.asarray(x, dtype=float))

def eval_phi_gegenbauer(x, n, lam=0.5):
    """Evaluate Gegenbauer C_n^{(lam)}(t) with t = 2*x - 1, x in [0,1]."""
    t = 2 * np.asarray(x, dtype=float) - 1
    return eval_gegenbauer(n, lam, t)

def gegenbauer_poly_coeffs(n, lam):
    """Monomial coefficients (ascending) of C_n^{(lam)}(t) in variable t in [-1,1]."""
    from scipy.special import gegenbauer as gegenbauer_poly
    C = gegenbauer_poly(n, lam)
    return C.c

def phi_gegenbauer_in_x(n, lam):
    """Express C_n(2x-1) as polynomial in x. scipy gegenbauer returns coeffs in descending power order."""
    c_t = gegenbauer_poly_coeffs(n, lam)
    max_k = len(c_t) - 1
    try:
        from scipy.special import comb
    except ImportError:
        from math import comb
    coeffs_x = np.zeros(max_k + 1)
    for k in range(max_k + 1):
        # scipy: c_t[0] t^max_k + c_t[1] t^{max_k-1} + ... so coeff of t^k is c_t[max_k - k]
        coef = c_t[max_k - k]
        if np.abs(coef) < 1e-15:
            continue
        for j in range(k + 1):
            sign = 1 if (k - j) % 2 == 0 else -1
            coeffs_x[j] += coef * comb(k, j) * (2 ** j) * sign
    return coeffs_x

## 4. Caputo derivative of basis functions

For each basis $\phi_n(x)$ we express it in monomials (in $x$ for Hermite; in $x$ after mapping for Gegenbauer) and apply $D^\alpha$ term-wise.

In [ ]:
def D_alpha_hermite(x, n, alpha):
    """Caputo D^alpha of Hermite H_n at x. x can be array."""
    x = np.atleast_1d(np.asarray(x, dtype=float))
    coeffs = hermite_poly_coeffs(n)
    out = np.zeros_like(x, dtype=float)
    for k in range(len(coeffs)):
        if np.abs(coeffs[k]) < 1e-14:
            continue
        out += coeffs[k] * caputo_monomial(x, k, alpha)
    return out

def D_alpha_gegenbauer(x, n, alpha, lam=0.5):
    """Caputo D^alpha of C_n(2x-1) at x in [0,1]."""
    x = np.atleast_1d(np.asarray(x, dtype=float))
    coeffs_x = phi_gegenbauer_in_x(n, lam)
    out = np.zeros_like(x, dtype=float)
    for k in range(len(coeffs_x)):
        if np.abs(coeffs_x[k]) < 1e-14:
            continue
        out += coeffs_x[k] * caputo_monomial(x, k, alpha)
    return out

## 5. Build basis and operational matrices at collocation points

$\Phi$: shape `(N_f, N)` with $\Phi_{i,n} = \phi_n(x_i)$.  
$D^\alpha \Phi$: shape `(N_f, N)` with $(D^\alpha \Phi)_{i,n} = D^\alpha \phi_n(x_i)$.  
Then $D^\alpha u \approx (D^\alpha \Phi) \, c$ with $c = (\Phi^T \Phi)^{-1} \Phi^T \, u_\theta(\mathbf{x})$.

In [ ]:
def build_basis_matrices(x_colloc, N_basis, alpha, basis="gegenbauer", gegenbauer_lam=0.5):
    """
    x_colloc: 1D array of collocation points.
    Returns: Phi (N_f x N), D_alpha_Phi (N_f x N).
    """
    x = np.asarray(x_colloc).ravel()
    N_f = len(x)
    Phi = np.zeros((N_f, N_basis))
    D_alpha_Phi = np.zeros((N_f, N_basis))
    for n in range(N_basis):
        if basis == "hermite":
            Phi[:, n] = eval_phi_hermite(x, n)
            D_alpha_Phi[:, n] = D_alpha_hermite(x, n, alpha)
        else:
            Phi[:, n] = eval_phi_gegenbauer(x, n, gegenbauer_lam)
            D_alpha_Phi[:, n] = D_alpha_gegenbauer(x, n, alpha, gegenbauer_lam)
    return Phi, D_alpha_Phi

## 6. Neural network and spectral projection

In [ ]:
class MLP(nn.Module):
    """Fully connected [1, 40, 40, 1] with tanh activations."""
    def __init__(self, layers=(1, 40, 40, 1)):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i + 1]))
        self.activation = torch.tanh

    def forward(self, x):
        h = x
        for i, layer in enumerate(self.layers[:-1]):
            h = self.activation(layer(h))
        return self.layers[-1](h)

def compute_coefficients(Phi, u_vals):
    """Least-squares: c = argmin ||Phi c - u||^2. u_vals shape (N_f,) or (N_f,1)."""
    u = np.asarray(u_vals).ravel()
    c, *_ = np.linalg.lstsq(Phi, u, rcond=None)
    return c

def D_alpha_u_spectral(u_at_colloc, Phi, D_alpha_Phi):
    """Given u_theta at collocation points, return D^alpha u at those points (numpy)."""
    c = compute_coefficients(Phi, u_at_colloc)
    return D_alpha_Phi @ c

def D_alpha_u_spectral_torch(u, Phi_t, D_alpha_Phi_t):
    """Differentiable: u (N_f,1), Phi_t (N_f, N), D_alpha_Phi_t (N_f, N). Returns (N_f,) so gradients flow."""
    # c = (Phi^T Phi)^{-1} Phi^T u
    PhiT = Phi_t.T
    A = PhiT @ Phi_t
    b = PhiT @ u
    c = torch.linalg.solve(A, b)
    return (D_alpha_Phi_t @ c).squeeze(-1)

## 7. Training loop (spectral-fPINN)

Loss: $\mathcal{L} = \omega_{res} \mathcal{L}_{res} + \omega_{bc} \mathcal{L}_{bc}$.

In [ ]:
def train_spectral_fPINN(
    alpha=0.75,
    lam=1.0,
    N_f=100,
    N_basis=15,
    basis="gegenbauer",
    gegenbauer_lam=0.5,
    epochs=2000,
    lr=1e-3,
    omega_res=1.0,
    omega_bc=40.0,
    layers=(1, 40, 40, 1),
    seed=42,
):
    torch.manual_seed(seed)
    np.random.seed(seed)
    x_colloc = np.linspace(0, 1, N_f, dtype=np.float32)
    Phi, D_alpha_Phi = build_basis_matrices(x_colloc, N_basis, alpha, basis, gegenbauer_lam)
    Phi_t = torch.tensor(Phi, dtype=torch.float32, device=device)
    D_alpha_Phi_t = torch.tensor(D_alpha_Phi, dtype=torch.float32, device=device)
    x_t = torch.tensor(x_colloc.reshape(-1, 1), dtype=torch.float32, device=device)
    x0 = torch.tensor([[0.0]], device=device)
    x1 = torch.tensor([[1.0]], device=device)
    model = MLP(layers).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    history = []
    t0 = time()
    for ep in range(epochs):
        model.train()
        opt.zero_grad()
        u = model(x_t)
        D_alpha_u = D_alpha_u_spectral_torch(u, Phi_t, D_alpha_Phi_t)
        residual = D_alpha_u + lam * torch.exp(u.squeeze(-1))
        L_res = (residual ** 2).mean()
        u0 = model(x0)
        u1 = model(x1)
        L_bc = (u0 ** 2).sum() + (u1 ** 2).sum()
        loss = omega_res * L_res + omega_bc * L_bc
        loss.backward()
        opt.step()
        history.append({"loss": loss.item(), "L_res": L_res.item(), "L_bc": L_bc.item()})
        if (ep + 1) % 400 == 0:
            print(f"Epoch {ep+1}/{epochs}  Loss = {loss.item():.6e}  L_res = {L_res.item():.6e}  L_bc = {L_bc.item():.6e}")
    wall_time = time() - t0
    return model, x_colloc, Phi, D_alpha_Phi, history, wall_time

## 8. L1 baseline (optional)

L1 scheme for Caputo: $D^\alpha u(x_n) \approx \frac{1}{\Gamma(2-\alpha) h^\alpha} \sum_{j=0}^{n-1} [u(x_{j+1})-u(x_j)]\,\big[(n-j)^{1-\alpha} - (n-j-1)^{1-\alpha}\big]$.

In [ ]:
def L1_caputo(u_on_grid, alpha, x_grid=None):
    """u_on_grid: values at uniformly spaced points (including endpoints)."""
    u = np.asarray(u_on_grid).ravel()
    n = len(u)
    if x_grid is None:
        h = 1.0 / (n - 1)
    else:
        h = (x_grid[-1] - x_grid[0]) / (n - 1)
    coeff = 1.0 / (gamma(2 - alpha) * (h ** alpha))
    D_alpha_u = np.zeros(n)
    for i in range(1, n):
        s = 0.0
        for j in range(i):
            diff = (i - j) ** (1 - alpha) - (i - j - 1) ** (1 - alpha)
            s += (u[j + 1] - u[j]) * diff
        D_alpha_u[i] = coeff * s
    return D_alpha_u

## 9. Reference solution: integer case ($\alpha=1$) and evaluation metrics

In [ ]:
def bratu_analytical_theta(lam):
    """Critical parameter theta for u(x) = -2*ln(cosh((x-0.5)*theta/2)/cosh(theta/4)) for classical Bratu."""
    from scipy.optimize import fsolve
    def eq(th):
        return th - np.sqrt(2 * lam) * np.cosh(th / 4)
    return float(fsolve(eq, 2.0)[0])

def bratu_exact_u(x, lam=1.0):
    """Analytical solution for classical Bratu u'' + lam*exp(u)=0, u(0)=u(1)=0."""
    x = np.atleast_1d(x)
    th = bratu_analytical_theta(lam)
    return -2 * np.log(np.cosh((x - 0.5) * th / 2) / np.cosh(th / 4))

def rel_l2_error(u_pred, u_exact):
    u_pred = np.asarray(u_pred).ravel()
    u_exact = np.asarray(u_exact).ravel()
    return np.linalg.norm(u_pred - u_exact) / (np.linalg.norm(u_exact) + 1e-14)

def max_error(u_pred, u_exact):
    return np.max(np.abs(np.asarray(u_pred).ravel() - np.asarray(u_exact).ravel()))

## 10. Run: Spectral-fPINN (Gegenbauer) for $\alpha=0.75$, $\lambda=1$

In [ ]:
model, x_colloc, Phi, D_alpha_Phi, history, wall_time = train_spectral_fPINN(
    alpha=0.75,
    lam=1.0,
    N_f=100,
    N_basis=15,
    basis="gegenbauer",
    gegenbauer_lam=0.5,
    epochs=2000,
    lr=1e-3,
    omega_res=1.0,
    omega_bc=40.0,
    seed=42,
)
print(f"Training time: {wall_time:.2f} s")

## 11. Predict and plot

In [ ]:
x_plot = np.linspace(0, 1, 200)
x_plot_t = torch.tensor(x_plot.reshape(-1, 1), dtype=torch.float32, device=device)
with torch.no_grad():
    u_pred = model(x_plot_t).cpu().numpy().ravel()

plt.figure(figsize=(6, 4))
plt.plot(x_plot, u_pred, "-", label="Spectral-fPINN (Gegenbauer)")
plt.xlabel("$x$")
plt.ylabel("$u(x)$")
plt.title("Fractional Bratu $\\alpha=0.75$, $\\lambda=1$")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 12. Compare with analytical solution for $\alpha=1$

In [ ]:
model_a1, x_c, Phi_a1, Da_Phi_a1, hist_a1, time_a1 = train_spectral_fPINN(
    alpha=1.0,
    lam=1.0,
    N_f=100,
    N_basis=15,
    basis="gegenbauer",
    epochs=2000,
    seed=42,
)

x_eval = np.linspace(0, 1, 200)
x_eval_t = torch.tensor(x_eval.reshape(-1, 1), dtype=torch.float32, device=device)
with torch.no_grad():
    u_p = model_a1(x_eval_t).cpu().numpy().ravel()
u_exact = bratu_exact_u(x_eval, 1.0)

print(f"Relative L2 error: {rel_l2_error(u_p, u_exact):.6e}")
print(f"Max error:         {max_error(u_p, u_exact):.6e}")

plt.figure(figsize=(6, 4))
plt.plot(x_eval, u_exact, "k-", label="Exact ($\\alpha=1$)")
plt.plot(x_eval, u_p, "--", label="Spectral-fPINN")
plt.xlabel("$x$")
plt.ylabel("$u(x)$")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 13. Solution profiles for different $\alpha$

In [ ]:
alphas = [0.5, 0.75, 1.0]
x_plot = np.linspace(0, 1, 200)
results = {}
for a in alphas:
    m, _, _, _, _, _ = train_spectral_fPINN(alpha=a, lam=1.0, N_basis=15, basis="gegenbauer", epochs=2000, seed=42)
    xt = torch.tensor(x_plot.reshape(-1, 1), dtype=torch.float32, device=device)
    with torch.no_grad():
        results[a] = m(xt).cpu().numpy().ravel()

u_exact = bratu_exact_u(x_plot, 1.0)
plt.figure(figsize=(6, 4))
for a in alphas:
    label = f"$\\alpha = {a}$" if a < 1 else f"$\\alpha = {a}$ (exact)"
    plt.plot(x_plot, results[a], label=label)
plt.plot(x_plot, u_exact, "k:", label="Exact $\\alpha=1$")
plt.xlabel("$x$")
plt.ylabel("$u(x)$")
plt.legend()
plt.title("Fractional Bratu: solution for different $\\alpha$")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 14. Hermite vs Gegenbauer (same $\alpha=1$)

In [ ]:
m_herm, _, _, _, _, t_herm = train_spectral_fPINN(alpha=1.0, lam=1.0, N_basis=15, basis="hermite", epochs=2000, seed=42)
m_geg,  _, _, _, _, t_geg  = train_spectral_fPINN(alpha=1.0, lam=1.0, N_basis=15, basis="gegenbauer", epochs=2000, seed=42)

x_eval = np.linspace(0, 1, 200)
u_exact = bratu_exact_u(x_eval, 1.0)
with torch.no_grad():
    u_herm = m_herm(torch.tensor(x_eval.reshape(-1, 1), dtype=torch.float32, device=device)).cpu().numpy().ravel()
    u_geg  = m_geg(torch.tensor(x_eval.reshape(-1, 1), dtype=torch.float32, device=device)).cpu().numpy().ravel()

print("Hermite:   Rel L2 =", rel_l2_error(u_herm, u_exact), "  Max err =", max_error(u_herm, u_exact), "  Time =", f"{t_herm:.2f}s")
print("Gegenbauer: Rel L2 =", rel_l2_error(u_geg, u_exact), "  Max err =", max_error(u_geg, u_exact), "  Time =", f"{t_geg:.2f}s")

plt.figure(figsize=(6, 4))
plt.plot(x_eval, u_exact, "k-", label="Exact")
plt.plot(x_eval, u_herm, "--", label="Spectral (Hermite)")
plt.plot(x_eval, u_geg, "-.", label="Spectral (Gegenbauer)")
plt.xlabel("$x$")
plt.ylabel("$u(x)$")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# RBF kernel and kernel matrices for LS-SVR
def rbf_kernel(x, y, gamma=10.0):
    """RBF (Gaussian) kernel K(x, y) = exp(-gamma * (x - y)^2). x, y can be arrays."""
    x, y = np.atleast_1d(x), np.atleast_1d(y)
    return np.exp(-gamma * (x - y) ** 2)

def build_kernel_matrices(x_grid, alpha, gamma=10.0):
    """
    x_grid: 1D array of collocation points (uniform grid including 0 and 1).
    Returns: K_mat (N x N), D_alpha_K (N x N).
    D_alpha_K[j, i] = (D^alpha of the function t -> K(t, x_i)) evaluated at x_j (via L1 scheme).
    """
    x = np.asarray(x_grid).ravel()
    N = len(x)
    K_mat = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            K_mat[j, i] = rbf_kernel(x[j], x[i], gamma)
    D_alpha_K = np.zeros((N, N))
    for i in range(N):
        # Column i: values of t -> K(t, x_i) at x_0,...,x_{N-1}
        u_col = K_mat[:, i].copy()
        D_alpha_K[:, i] = L1_caputo(u_col, alpha, x_grid=x)
    return K_mat, D_alpha_K

In [ ]:
def lssvr_predict(x_eval, x_colloc, alpha, gamma=10.0):
    """Evaluate LS-SVR solution u(x) = sum_i alpha_i K(x, x_i) at x_eval."""
    x_eval = np.atleast_1d(x_eval)
    x_colloc = np.asarray(x_colloc).ravel()
    u = np.zeros_like(x_eval, dtype=float)
    for i, x in enumerate(x_eval):
        u[i] = np.sum(alpha * rbf_kernel(x, x_colloc, gamma))
    return u